## IMPORTS

In [1]:
import warnings
#warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import time

In [2]:
%matplotlib inline

In [3]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.6 MB/s eta 0:00:00


In [4]:
from keras.layers import Input, LSTM, Dense, concatenate
from keras.models import Model
from transformers import BertTokenizer, TFBertModel
from keras.layers import Input, Dense
from keras.models import Model

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

MAX_LENGTH = 512 #can be changed bruh

input_ids = Input(shape=(MAX_LENGTH,), dtype='int32', name='input_ids')
attention_mask = Input(shape=(MAX_LENGTH,), dtype='int32', name='attention_mask')

bert_output = bert_model([input_ids, attention_mask])
# Here, we're using the [CLS] token's representation as the sequence representation
sequence_output = bert_output.last_hidden_state[:, 0, :]

# Add the classification layer
sentiment_output = Dense(1, activation='sigmoid')(sequence_output)  # Binary classification

sentiment_model = Model(inputs=[input_ids, attention_mask], outputs=sentiment_output)
sentiment_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

## Calendar Preprocess

In [38]:
# Read the data from csv file

#NOTE: in final code replace the csv file with the input from the website
calendar_df = pd.read_csv("Punit_calendar.csv")
calendar_df = calendar_df.rename(columns={"Given planned earliest start": "start", "Given planned earliest end": "end"})
print(len(calendar_df))
calendar_df.head(10)

141


,Title,start,end,Notes,Assigned Resources,Additional Title
0,Signals exam next week,09/12/2022 07:00 AM CDT,09/12/2022 08:00 AM CDT,NaN,NaN,NaN
1,Signals Exam next week,10/10/2022 07:00 AM CDT,10/10/2022 08:00 AM CDT,NaN,NaN,NaN
2,Signals exam next week,11/07/2022 07:15 AM CST,11/07/2022 08:00 AM CST,NaN,NaN,NaN
3,Signals final next week,12/05/2022 06:00 AM CST,12/05/2022 07:00 AM CST,NaN,NaN,NaN
4,Physio Exam next week,09/15/2022 07:00 AM CDT,09/15/2022 08:00 AM CDT,NaN,NaN,NaN
5,Physio exam next week,10/27/2022 06:00 AM CDT,10/27/2022 07:00 AM CDT,NaN,NaN,NaN
6,Physio Exam Next week,11/24/2022 06:00 AM CST,11/24/2022 07:00 AM CST,NaN,NaN,NaN
7,Physio (Brock) OH,08/19/2022 01:00 PM CDT,08/19/2022 02:00 PM CDT,NaN,NaN,NaN
8,Bjj,08/30/2022 05:00 PM CDT,08/30/2022 07:00 PM CDT,NaN,NaN,NaN
9,Bjj,09/01/2022 05:00 PM CDT,09/01/2022 07:00 PM CDT,NaN,NaN,NaN


### A. Data prep

In [39]:
#From the csv file, create another calendar_dfframe that parses events per day. and generates features
from datetime import datetime
calendar_df[['start_date', 'start_time']] = calendar_df['start'].str.split(' ', 1, expand=True)
calendar_df['start_time'] = calendar_df['start_time'].str.slice(0,5)
calendar_df['start_time'] = calendar_df['start_time'].astype(str)
calendar_df[['end_date', 'end_time']] = calendar_df['end'].str.split(' ', 1, expand=True)
calendar_df['end_time'] = calendar_df['end_time'].str.slice(0,5)
calendar_df['end_time'] = calendar_df['end_time'].astype(str)
# Function to convert time strings to total minutes
def convert_to_minutes(time_str):
    if(time_str == 'nan'):
      return 540
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

# Apply the function to the column
calendar_df['total_minutes_start'] = calendar_df['start_time'].apply(convert_to_minutes)
calendar_df['total_minutes_end'] = calendar_df['end_time'].apply(convert_to_minutes)
calendar_df['duration'] = calendar_df['total_minutes_end'].sub(calendar_df['total_minutes_start'])
calendar_df['duration'] = calendar_df['duration'].abs()
calendar_df = calendar_df.drop(columns=['start_time', 'end_time'])
print(calendar_df.head(10))




                     Title                    start                      end  \
0   Signals exam next week  09/12/2022 07:00 AM CDT  09/12/2022 08:00 AM CDT   
1   Signals Exam next week  10/10/2022 07:00 AM CDT  10/10/2022 08:00 AM CDT   
2   Signals exam next week  11/07/2022 07:15 AM CST  11/07/2022 08:00 AM CST   
3  Signals final next week  12/05/2022 06:00 AM CST  12/05/2022 07:00 AM CST   
4    Physio Exam next week  09/15/2022 07:00 AM CDT  09/15/2022 08:00 AM CDT   
5    Physio exam next week  10/27/2022 06:00 AM CDT  10/27/2022 07:00 AM CDT   
6    Physio Exam Next week  11/24/2022 06:00 AM CST  11/24/2022 07:00 AM CST   
7        Physio (Brock) OH  08/19/2022 01:00 PM CDT  08/19/2022 02:00 PM CDT   
8                      Bjj  08/30/2022 05:00 PM CDT  08/30/2022 07:00 PM CDT   
9                      Bjj  09/01/2022 05:00 PM CDT  09/01/2022 07:00 PM CDT   

  Notes Assigned Resources Additional Title  start_date    end_date  \
0   NaN                NaN              NaN  09/

<ipython-input-39-1b6646d403f4>:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  calendar_df[['start_date', 'start_time']] = calendar_df['start'].str.split(' ', 1, expand=True)
<ipython-input-39-1b6646d403f4>:6: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  calendar_df[['end_date', 'end_time']] = calendar_df['end'].str.split(' ', 1, expand=True)


In [40]:
#add a repeat column for repeating events
calendar_df['repeating'] = calendar_df['Notes'].str.slice(0,4)
calendar_df['repeating'] = calendar_df['repeating'].apply(lambda x: 1 if x == 'FREQ' else 0)
try:
  calendar_df = calendar_df.drop(columns=['Notes', 'Assigned Resources', 'Additional Title', 'start', 'end'])
except:
  print("error dropping")
print(calendar_df.head(10))


                     Title  start_date    end_date  total_minutes_start  \
0   Signals exam next week  09/12/2022  09/12/2022                  420   
1   Signals Exam next week  10/10/2022  10/10/2022                  420   
2   Signals exam next week  11/07/2022  11/07/2022                  435   
3  Signals final next week  12/05/2022  12/05/2022                  360   
4    Physio Exam next week  09/15/2022  09/15/2022                  420   
5    Physio exam next week  10/27/2022  10/27/2022                  360   
6    Physio Exam Next week  11/24/2022  11/24/2022                  360   
7        Physio (Brock) OH  08/19/2022  08/19/2022                   60   
8                      Bjj  08/30/2022  08/30/2022                  300   
9                      Bjj  09/01/2022  09/01/2022                  300   

   total_minutes_end  duration  repeating  
0                480        60          0  
1                480        60          0  
2                480        45          0 

In [41]:
from pandas.core.frame import DataFrame
from numpy.core.multiarray import datetime_as_string
# Sort by ascending date
# Convert the date_column to datetime objects
calendar_df['date_column_start'] = pd.to_datetime(calendar_df['start_date'], format='%m/%d/%Y')
calendar_df['date_column_end'] = pd.to_datetime(calendar_df['end_date'], format='%m/%d/%Y')
# Sort the calendar_dfFrame based on the date_column
calendar_df = calendar_df.sort_values(by='date_column_start')

# Reset the index if needed
calendar_df = calendar_df.reset_index(drop=True)
#drop last row as it does not contain any useful calendar_df
calendar_df = calendar_df.drop([len(calendar_df)-1])
# Remove rows where all data is missing or NaN
calendar_df = calendar_df.dropna(how='all')
print(calendar_df.tail(10))
print(len(calendar_df))


                Title  start_date    end_date  total_minutes_start  \
130  Capstone meeting  09/25/2023  09/25/2023                  720   
131               Gym  09/26/2023  09/26/2023                  660   
132   Gym+sauna +swim  09/27/2023  09/27/2023                  120   
133               Gym  09/28/2023  09/28/2023                  660   
134          Climbing  09/29/2023  09/29/2023                  630   
135           Workout  09/30/2023  09/30/2023                   60   
136   networking exam  10/05/2023  10/05/2023                  420   
137         PCL 4.429  10/06/2023  10/06/2023                  600   
138         Paintball  10/14/2023  10/14/2023                  615   
139         hackathon  10/21/2023  10/21/2023                  540   

     total_minutes_end  duration  repeating date_column_start date_column_end  
130                 60       660          0        2023-09-25      2023-09-25  
131                735        75          0        2023-09-26      20

## Spending Preprocess

In [42]:
# Read the data from csv file

#NOTE: in final code replace the csv file with the input from the website
spending_df = pd.read_csv("Spendings.csv")
print(len(spending_df))
spending_df.head(10)

958


,Date,Name,Amount
0,1/12/2022,DING TEA,$14.51
1,1/12/2022,TACO BELL,$14.03
2,1/18/2022,CLUTCH PREP,$39.99
3,1/18/2022,TROPICAL SMOOTHIE CAFE,$6.48
4,1/19/2022,BOBA,$5.20
5,1/21/2022,KROGER,$8.07
6,1/22/2022,BUC-EE'S,$5.40
7,1/23/2022,7-ELEVEN,$3.89
8,1/25/2022,TST* RAMEN TATSU-YA,$16.24
9,1/28/2022,CHIPOTLE,$8.23


### A. Data prep

In [43]:
from pandas.core.frame import DataFrame
from numpy.core.multiarray import datetime_as_string

# Convert the date_column to datetime objects
spending_df['date_column_start'] = pd.to_datetime(spending_df['Date'], format='%m/%d/%Y')

# Remove rows where all data is missing or NaN
spending_df = spending_df.dropna(how='all')

# Sort the DataFrame based on the date_column
spending_df = spending_df.sort_values(by='date_column_start')

# Reset the index if needed
spending_df = spending_df.reset_index(drop=True)

# Drop the last row if it does not contain any useful data
spending_df = spending_df.drop([len(spending_df)-1])

print(spending_df.head(10))
print(len(spending_df))


        Date                     Name  Amount date_column_start
0  1/12/2022               DING TEA    $14.51        2022-01-12
1  1/12/2022              TACO BELL    $14.03        2022-01-12
2  1/18/2022            CLUTCH PREP    $39.99        2022-01-18
3  1/18/2022  TROPICAL SMOOTHIE CAFE    $6.48        2022-01-18
4  1/19/2022                  BOBA      $5.20        2022-01-19
5  1/21/2022                KROGER      $8.07        2022-01-21
6  1/22/2022              BUC-EE'S      $5.40        2022-01-22
7  1/23/2022              7-ELEVEN      $3.89        2022-01-23
8  1/25/2022     TST* RAMEN TATSU-YA   $16.24        2022-01-25
9  1/28/2022              CHIPOTLE      $8.23        2022-01-28
455


## MODEL

In [46]:
# BERT embeddings input for past 10 events
bert_past_event_input = Input(shape=(10, 768), name='bert_past_event_input')
x1_past_event = LSTM(128, return_sequences=True)(bert_past_event_input)
x1_past_event = LSTM(64)(x1_past_event)

# Structured data input (dates, durations, repeats for past 10 events)
struct_past_event_input = Input(shape=(10, 3), name='struct_past_event_input')
x2_past_event = LSTM(64, return_sequences=True)(struct_past_event_input)
x2_past_event = LSTM(32)(x2_past_event)

# Past spending vendor BERT
bert_past_spending_input = Input(shape=(10, 768), name='bert_past_spending_input')
x4_past_spending = LSTM(128, return_sequences=True)(bert_past_spending_input)
x4_past_spending = LSTM(64)(x4_past_spending)

# Past spending data: Date, Amount
struct_past_spending_input = Input(shape=(10, 2), name='struct_past_spending_input')  # I assume this should be (10, 2) and not (10, 3) as described
x5_past_spending = LSTM(64, return_sequences=True)(struct_past_spending_input)
x5_past_spending = LSTM(32)(x5_past_spending)

# BERT embeddings input for next 10 events
bert_future_input = Input(shape=(10, 768), name='bert_future_input')
x1_future = LSTM(128, return_sequences=True)(bert_future_input)
x1_future = LSTM(64)(x1_future)

# Structured data input (dates, durations, repeats for next 10 events)
struct_future_input = Input(shape=(10, 3), name='struct_future_input')
x2_future = LSTM(64, return_sequences=True)(struct_future_input)
x2_future = LSTM(32)(x2_future)

# Current date input (assuming encoded as a single value)
current_date_input = Input(shape=(1,), name='current_date_input')

# Merging all processed inputs
merged = concatenate([x1_past_event, x2_past_event, x4_past_spending, x5_past_spending, x1_future, x2_future, current_date_input])

# Continue processing the merged data
output = Dense(128, activation='relu')(merged)
output = Dense(1, activation='linear')(output)  # Assuming a regression problem for spending prediction

model = Model(inputs=[bert_past_event_input, struct_past_event_input, bert_past_spending_input, struct_past_spending_input, bert_future_input, struct_future_input, current_date_input],
              outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')



In [47]:
print(calendar_df)
print(spending_df)


                 Title  start_date    end_date  total_minutes_start  \
0    Physio (Brock) OH  08/19/2022  08/19/2022                   60   
1    Transport Erik OH  08/22/2022  08/22/2022                  180   
2          EMS meeting  08/25/2022  08/25/2022                  420   
3       Advance Austin  08/30/2022  08/30/2022                   60   
4                  Bjj  08/30/2022  08/30/2022                  300   
..                 ...         ...         ...                  ...   
135            Workout  09/30/2023  09/30/2023                   60   
136    networking exam  10/05/2023  10/05/2023                  420   
137          PCL 4.429  10/06/2023  10/06/2023                  600   
138          Paintball  10/14/2023  10/14/2023                  615   
139          hackathon  10/21/2023  10/21/2023                  540   

     total_minutes_end  duration  repeating date_column_start date_column_end  
0                  120        60          0        2022-08-19      

In [48]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Tokenize event names and vendor names
#calendar_df['event_tokens'] = calendar_df['Title'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))
#spending_df['vendor_tokens'] = spending_df['Name'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))
MAX_TOKENS = 768

def pad_or_truncate(tokens):
    if len(tokens) < MAX_TOKENS:
        return tokens + [0] * (MAX_TOKENS - len(tokens))
    return tokens[:MAX_TOKENS]

calendar_df['event_tokens'] = calendar_df['Title'].apply(lambda x: pad_or_truncate(tokenizer.encode(x, add_special_tokens=True, truncation=True)))
spending_df['vendor_tokens'] = spending_df['Name'].apply(lambda x: pad_or_truncate(tokenizer.encode(x, add_special_tokens=True, truncation=True)))


# Determine the date interval
start_date = calendar_df['date_column_start'].iloc[9]  # 10th event
end_date = calendar_df['date_column_start'].iloc[-10]  # 10th to last event
date_range = pd.date_range(start=start_date, end=end_date)


# Helper function to get the past and future 10 events/spendings and spending for next week
def get_data(date):
    # Check if 'Amount' is of type string, if not, convert it
    if spending_df['Amount'].dtype != 'object':
        spending_df['Amount'] = spending_df['Amount'].astype(str)

    # Convert 'Amount' column to float
    spending_df['Amount'] = spending_df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)

    past_events = calendar_df[calendar_df['date_column_start'] < date].tail(10)
    future_events = calendar_df[calendar_df['date_column_start'] > date].head(10)
    past_spendings = spending_df[spending_df['date_column_start'] < date].tail(10)

    # Now sum up 'Amount' after conversion
    next_week_spending = spending_df[(spending_df['date_column_start'] > date) & (spending_df['date_column_start'] <= date + timedelta(days=7))].sum()['Amount']

    return past_events, future_events, past_spendings, next_week_spending


# Prepare the training data
X_train_past_event_tokens = []
X_train_past_event_structured = []
X_train_past_spending_tokens = []
X_train_past_spending_structured = []
X_train_future_event_tokens = []
X_train_future_event_structured = []
X_train_current_date_encoded = []
y_train = []

for date in date_range:
    past_events, future_events, past_spendings, next_week_spending = get_data(date)

    if len(past_events) >= 10 and len(future_events) >= 10:

      past_events['date_column_start'] = past_events['date_column_start'].apply(lambda x: x.timestamp())
      future_events['date_column_start'] = future_events['date_column_start'].apply(lambda x: x.timestamp())
      past_spendings['date_column_start'] = past_spendings['date_column_start'].apply(lambda x: x.timestamp())

      # Append BERT tokens for past and future events and past spending
      X_train_past_event_tokens.append(np.array(past_events['event_tokens'].tolist()))
      X_train_past_spending_tokens.append(np.array(past_spendings['vendor_tokens'].tolist()))
      X_train_future_event_tokens.append(np.array(future_events['event_tokens'].tolist()))

      # Append structured data for past and future events and past spending
      X_train_past_event_structured.append(np.array(past_events[['date_column_start', 'repeating', 'duration']]))
      X_train_past_spending_structured.append(np.array(past_spendings[['date_column_start', 'Amount']]))
      X_train_future_event_structured.append(np.array(future_events[['date_column_start', 'repeating', 'duration']]))

      # Append the current date (encoded as a single value)
      X_train_current_date_encoded.append([date.timestamp()])
      y_train.append(next_week_spending)

# Convert lists to arrays
X_train_past_event_tokens = np.array(X_train_past_event_tokens)
X_train_past_event_structured = np.array(X_train_past_event_structured)
X_train_past_spending_tokens = np.array(X_train_past_spending_tokens)
X_train_past_spending_structured = np.array(X_train_past_spending_structured)

X_train_future_event_tokens = np.array(X_train_future_event_tokens)
X_train_future_event_structured = np.array(X_train_future_event_structured)
X_train_current_date_encoded = np.array(X_train_current_date_encoded)
y_train = np.array(y_train)


<ipython-input-48-1cf041f91410>:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  spending_df['Amount'] = spending_df['Amount'].str.replace('$', '').str.replace(',', '').astype(float)
<ipython-input-48-1cf041f91410>:39: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  next_week_spending = spending_df[(spending_df['date_column_start'] > date) & (spending_df['date_column_start'] <= date + timedelta(days=7))].sum()['Amount']
<ipython-input-48-1cf041f91410>:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *no

In [49]:
print(X_train_past_event_tokens.dtype)
print(X_train_past_event_structured.dtype)
print(X_train_past_spending_tokens.dtype)
print(X_train_past_spending_structured.dtype)
print(X_train_future_event_tokens.dtype)
print(X_train_future_event_structured.dtype)
print(X_train_current_date_encoded.dtype)
print(y_train.dtype)

int64
float64
int64
float64
int64
float64
float64
float64


In [50]:
print(y_train)


[2.67700e+01 2.67700e+01 2.67700e+01 2.67700e+01 2.53800e+01 1.53800e+01
 8.61000e+00 8.61000e+00 1.53800e+01 2.77700e+01 2.77700e+01 2.75500e+01
 2.75500e+01 3.16400e+01 3.49200e+01 4.94800e+01 3.70900e+01 4.79100e+01
 4.81300e+01 6.88800e+01 6.47900e+01 8.61000e+01 8.20800e+01 8.20800e+01
 7.12600e+01 7.12600e+01 5.05100e+01 6.55100e+01 4.09200e+01 6.82600e+01
 6.82600e+01 7.66500e+01 7.68600e+01 1.51860e+02 2.07330e+02 2.07330e+02
 3.12070e+02 3.39250e+02 3.39250e+02 3.30430e+02 2.75720e+02 2.05250e+02
 2.05250e+02 1.09540e+02 8.23600e+01 9.39700e+01 9.89000e+01 7.86100e+01
 1.16080e+02 1.51780e+02 1.11630e+02 1.11630e+02 9.16300e+01 1.04010e+02
 1.20240e+02 8.27700e+01 4.70700e+01 4.35400e+01 4.35400e+01 4.35400e+01
 2.62300e+01 1.00000e+01 1.75600e+01 7.02100e+01 6.02100e+01 6.83300e+01
 7.62900e+01 7.62900e+01 7.62900e+01 6.87300e+01 7.19000e+01 7.19000e+01
 6.37800e+01 5.58200e+01 5.58200e+01 5.58200e+01 1.38240e+02 8.24200e+01
 8.24200e+01 8.24200e+01 8.24200e+01 9.93900e+01 9.

In [51]:
# Train the model
model_input = [X_train_past_event_tokens, X_train_past_event_structured, X_train_past_spending_tokens, X_train_past_spending_structured, X_train_future_event_tokens, X_train_future_event_structured, X_train_current_date_encoded]
model.fit(model_input, y_train, epochs=20, batch_size=32)

Epoch 1/20
13/13 [==============================] - 29s 148ms/step - loss: 56578828926976.0000
Epoch 2/20
13/13 [==============================] - 2s 154ms/step - loss: 13995261034496.0000
Epoch 3/20
13/13 [==============================] - 3s 260ms/step - loss: 2962677301248.0000
Epoch 4/20
13/13 [==============================] - 2s 179ms/step - loss: 816858791936.0000
Epoch 5/20
13/13 [==============================] - 2s 137ms/step - loss: 235004100608.0000
Epoch 6/20
13/13 [==============================] - 2s 139ms/step - loss: 61938966528.0000
Epoch 7/20
13/13 [==============================] - 2s 137ms/step - loss: 17235156992.0000
Epoch 8/20
13/13 [==============================] - 2s 146ms/step - loss: 4942908416.0000
Epoch 9/20
13/13 [==============================] - 2s 151ms/step - loss: 1264749952.0000
Epoch 10/20
13/13 [==============================] - 3s 260ms/step - loss: 288184832.0000
Epoch 11/20
13/13 [==============================] - 3s 190ms/step - loss: 7876333